In [1]:

from langchain_groq import ChatGroq

In [4]:

llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama3-8b-8192"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to walk on the Moon was Neil Armstrong, an American astronaut who stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the Moon.


In [8]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://careers.nike.com/director-software-engineering-itc/job/R-49416")
page_data = loader.load().pop().page_content
print(page_data)


USER_AGENT environment variable not set, consider setting it to identify your requests.























Director Software Engineering, ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [10]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res


[{'role': 'Director Software Engineering, ITC',
  'experience': {'years': 14, 'leadership': 5, 'cloud native technologies': 5},
  'skills': ['Excellent verbal and written communication skills, both technical and business-facing',
   'Strength in strategic thinking and translating strategy to measurable objectives',
   'Track record of inspiring others & positive change in solution design, technical delivery, and process improvement',
   'Passion for learning new technologies, patterns, and practices',
   'Experience with Agile software development methodology and DevOps (CICD)',
   'Experience with deploying, monitoring and tracing solutions using tools like Splunk, SignalFx and New Relic',
   'Fluent in modern software architectural principles and patterns (e.g. REST, domain-driven design, microservices, etc.)',
   'Experience with current front-end web application tech (e.g. javascript, CSS, html5, Vue, React/redux)',
   'Experience with distributed cloud systems tech (e.g. node.js, 

In [11]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [12]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionAddEvent: capture() takes 1 positional argument but 3 were given


In [19]:

job = json_res
job

[{'role': 'Director Software Engineering, ITC',
  'experience': {'years': 14, 'leadership': 5, 'cloud native technologies': 5},
  'skills': ['Excellent verbal and written communication skills, both technical and business-facing',
   'Strength in strategic thinking and translating strategy to measurable objectives',
   'Track record of inspiring others & positive change in solution design, technical delivery, and process improvement',
   'Passion for learning new technologies, patterns, and practices',
   'Experience with Agile software development methodology and DevOps (CICD)',
   'Experience with deploying, monitoring and tracing solutions using tools like Splunk, SignalFx and New Relic',
   'Fluent in modern software architectural principles and patterns (e.g. REST, domain-driven design, microservices, etc.)',
   'Experience with current front-end web application tech (e.g. javascript, CSS, html5, Vue, React/redux)',
   'Experience with distributed cloud systems tech (e.g. node.js, 

In [21]:
links = collection.query(query_texts=job[0]['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/rea

In [22]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhancing Nike's Product Innovation and Consumer Creation with AtliQ's Expertise

Dear [Director Software Engineering, ITC],

I came across the job description for the Director Software Engineering, ITC role at Nike, and I was impressed by the requirements. As a business development executive at AtliQ, I'd like to introduce our company's capabilities in fulfilling your needs.

AtliQ is an AI & Software Consulting company dedicated to facilitating the seamless integration of business processes through automated tools. Our expertise lies in empowering enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency.

Our portfolio showcases our proficiency in various technologies, including cloud native technologies, Agile software development methodology, DevOps, and experience with deploying, monitoring, and tracing solutions using tools like Splunk, SignalFx, and New Relic. We have successfully delivered proje